In [1]:
from itertools import combinations 

In [2]:
# for testing purposes
items = [1, 2, 3]
expected = [(1), (1, 2), (1, 3), (1, 2, 3), (2), (2, 3), (1, )]

In [3]:
def powerset_without_emptyset(items):
    '''
    Returns the powerset of a list of items as a list of tuples, excluding the empty set
    '''
    combos = []
    for i in range(len(items)):
        combos.extend(list(combinations(items, len(items) - i)))
    return combos


In [4]:
my_set = powerset_without_emptyset(items)
my_set

[(1, 2, 3), (1, 2), (1, 3), (2, 3), (1,), (2,), (3,)]

In [8]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import cv2 
import skimage
import numpy
from torchvision import datasets, transforms
from PIL import Image

Hu moments are a set of 7 numbers calculated using central moments that are invariant to image transformations. The first 6 moments have been proved to be invariant to translation, scale, and rotation, and reflection. While the 7th moment’s sign changes for image reflection.

In [13]:
transforms_to_use = [cv2.HuMoments, skimage.feature.graycomatrix, 
                     cv2.calcHist, skimage.feature.local_binary_pattern]

dataset = datasets.OxfordIIITPet('/data', download=True, transform=None) # a dataset of PIL images 

# getting a pil image
img, label = dataset.__getitem__(0)
# Convert the PIL image to a NumPy array and then to grayscale for use with OpenCV functions
im = numpy.array(img)
if im.ndim == 3:
    im_gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
else:
    im_gray = im

# Compute Hu Moments
hu_moments = cv2.HuMoments(cv2.moments(im_gray)).flatten()
hu_moments

array([ 1.27659284e-03,  1.77966539e-07,  9.13967781e-11,  3.09407880e-11,
        1.15955047e-21, -7.09286670e-15,  1.16733530e-21])

In [19]:
# Example: compute gray-level co-occurrence matrix (GLCM) for im_gray
dists = [1]  # pixel pair distance
angles = [0] # angle in radians
glcm = skimage.feature.graycomatrix(im_gray, distances=dists, angles=angles, symmetric=True, normed=True)

# Select the first distance and angle to get a 2D matrix
glcm_2d = glcm[:, :, 0, 0]

# Normalize to 0-255 for visualization
glcm_img = (255 * (glcm_2d - glcm_2d.min()) / (glcm_2d.ptp() + 1e-8)).astype('uint8')
Image.fromarray(glcm_img).show()

In [ ]:
# Compute histogram for original image
hist = cv2.calcHist([im], [0], None, [256], [0, 256])
Image.fromarray(hist).show()

In [ ]:
# Compute Local Binary Pattern with 8 points and radius 1 (common choices)
lbp = skimage.feature.local_binary_pattern(im_gray, P=8, R=1)
Image.fromarray(lbp).show() # super cool

In [ ]:
from sklearn.metrics.pairwise import nan_euclidean_distances

# TODO: check if this works - idk if the logic is right
def highest_separability(transformations, arrays):
    '''
    Parameters: 
        transformations - a list containing the transformations you want to try (e.g. [cv2.calcHist, skimage.feature.local_binary_pattern])
        arrays - [batch, h, w, channels] a list of normalized image arrays

    Returns:
        the transformation or combination of transformations (as a tuple) that 
        produces the most class separability
    '''

    combos = powerset_without_emptyset(transformations)
    best_combo = None
    best_score = -float('inf')

    for combo in combos:
        # Apply each transformation in the combo sequentially to all images
        transformed = []
        for img in arrays:
            temp = img
            for t in combo:
                temp = t(temp)
            transformed.append(temp)
        
        # Flatten features for separability calculation
        features = [f.flatten() for f in transformed]
        
        # Compute separability (e.g., Fisher score or another metric)
        # Here, assume you have a function `compute_separability(features, labels)`
        # You may need to pass labels if available
        score = nan_euclidean_distances(...) # TODO: I need to compare X and Y, but I only have `features`
        
        if score > best_score:
            best_score = score
            best_combo = combo

    return best_combo, best_score